**The next few code blocks would just be repetition of a few steps from the original notebook,<br>
Please skip to continue from the new additions**

In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("/kaggle/input/aihsentiment/AI-Hackathon-test-data-set-3.csv")
pos_data = data[data["Sentiment"] == 1]
neg_data = data[data["Sentiment"] == -1]
net_data = data[data["Sentiment"] == 0]
use_pos = pos_data[:4000]
use_neg = neg_data[:4000]
data1 = pd.concat([use_pos, use_neg, net_data])
from sklearn.utils import shuffle
data1 = shuffle(data1, random_state = 42)
x = data1["Comments"]
y = data1["Sentiment"]

x_train = x[:int(len(x)*0.75)]
y_train = y[:int(len(y)*0.75)]
x_test = x[int(len(x)*0.75):]
y_test = y[int(len(y)*0.75):]
from transformers import BertTokenizer as BT, TFBertForSequenceClassification as tbsc
from sklearn.metrics import classification_report as cr
tokenizer = BT.from_pretrained('bert-base-uncased', do_lower_case=True)

x_train_en = tokenizer.batch_encode_plus(x_train.astype(str).tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = 128,
                                              return_tensors='tf')
 
x_test_en= tokenizer.batch_encode_plus(x_test.astype(str).tolist(),
                                              padding=True, 
                                              truncation=True,
                                              max_length = 128,
                                              return_tensors='tf')
model = tbsc.from_pretrained('bert-base-uncased', num_labels=3)
y_train = y_train + 1
y_test = y_test + 1
import tensorflow as tf
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**This is where the continuation of the notebook starts from**<br><br>
We start with the model training

In [2]:
model.fit(
    [x_train_en['input_ids'], x_train_en['token_type_ids'], x_train_en['attention_mask']],
    y_train,
    validation_data=(
      [x_test_en['input_ids'], x_test_en['token_type_ids'], x_test_en['attention_mask']],y_test),
    batch_size=32,
    epochs=3
)

Epoch 1/3
286/286 [==============================] - 70s 109ms/step - loss: 0.2305 - accuracy: 0.9387 - val_loss: 0.1629 - val_accuracy: 0.9524
Epoch 2/3
286/286 [==============================] - 28s 99ms/step - loss: 0.1464 - accuracy: 0.9594 - val_loss: 0.1573 - val_accuracy: 0.9573
Epoch 3/3
286/286 [==============================] - 29s 102ms/step - loss: 0.1458 - accuracy: 0.9578 - val_loss: 0.1622 - val_accuracy: 0.9524


As the evaluation shows, our model gains an accuracy of about ~96%.<br>
Lets evaluate our model on the test data.

In [3]:
test_loss, test_accuracy = model.evaluate(
    [x_test_en['input_ids'], x_test_en['token_type_ids'], x_test_en['attention_mask']],
    y_test
)
print(f'Test loss: {test_loss}, Test accuracy: {test_accuracy}')

96/96 [==============================] - 3s 32ms/step - loss: 0.1622 - accuracy: 0.9524
Test loss: 0.1621810644865036, Test accuracy: 0.9523965716362


As we can see, our model performs well on the test data as well.<br>
We can proceed with the use of this model for our goal.<br>

### Section 4: User interactive space

Here, we allow the model to be used for different data.<br>

**Note**: The performance of this model on real world data is extremely poor,<br>
due to the source dataset having effectively only 22 data points.<br>
As the motive of the hackathon's organizers was to test the preprocessing abilities of the participants, this model's performance is relevant in the scope of evaluating against the dataset itself.

In [4]:
while True:
    print("Enter the array of reviews you want to predict.")
    arr = list(input().rstrip().split(", "))
    df = pd.DataFrame(arr, columns=['Comments'])
    df_en = tokenizer.batch_encode_plus(x_train.astype(str).tolist(),
                                        padding=True, 
                                        truncation=True,
                                        max_length = 128,
                                        return_tensors='tf')
    pred = model.predict(
        [df_en['input_ids'],df_en['token_type_ids'], df_en['attention_mask']])
    logits = pred.logits
    pred_labels = tf.argmax(logits, axis=1)
    pred_labels = pred_labels.numpy()
    label = {
        2: 'Positive',
        0: 'Negative',
        1: 'Neutral'
    }
    pred_labels = [label[i] for i in pred_labels]
    print("Predicted sentiments are - \n")
    for i in range(len(arr)):
        print(arr[i],":",pred_labels[i])
    print("Do you wish to continue?\n1 - Yes\n2 - No")
    if int(input()) == 2:
        break

print("Thank you.")

    

Enter the array of reviews you want to predict.


 Good product


286/286 [==============================] - 12s 33ms/step
Predicted sentiments are - 

Good product : Positive
Do you wish to continue?
1 - Yes
2 - No


 2


Thank you.


This marks the end of our project.

## Conclusion

This project is aimed at determining the sentiment of reviews of products accurately.<br>
The true core challenge of the project was to counter the challenges faced by having a poor dataset.<br>
Challenges faces in the dataset -
- Heavy redundancy (>98%), causing no real input value in the dataset (the dataset had 22 rows post dropping duplicate values, which originally had 20000 rows)
- Wrong, with low variety in data in the first place, different instances of identical features having different label added to the loss of value in the data<br>

Due to the above reasons (very low variety in data), the model trained on the same is essentially not of practical use.<br><br>
With the challenge being to make the model accurate for data within the dataset itself,<br>
our model gained an accuracy of ~96%, which originally hit an bottleneck accuracy of 60% prior to all preprocessing ideas being implemented.
With that, the goal has been met & the project is complete.

**What makes this project different (& better)?**
-> Accuracy of ~95% compared to the bottleneck average of 60%<br><br>
What was the main factor behind the accuracy increase?<br>
Resolving massive data annotation errors in the original dataset.<br><br>
Due to the inconsistent annotation, fitting the model with the entire dataset couldn't reproduce a high accuracy. The reason being the same data being present multiple times, with different annotations.